## Structs and Functions

*From Chapter 16. Structs and Functions - Think Julia, by Ben Lauwens*  
*From Chapter 17. Multiple Dispatch  - Think Julia, by Ben Lauwens*  

Resources:
- https://github.com/BenLauwens/ThinkJulia.jl


In [1]:
"""
Represents the time of day.

fields: hour, minute, second
"""
struct MyTime
    hour::Int64
    minute::Int64
    second::Int64
    
    # inner constructor, to enforce invariants:
    function MyTime(hour::Int64=0, minute::Int64=0, second::Int64=0)
        @assert(0 ≤ minute < 60, "Minute is not between 0 and 60.")
        @assert(0 ≤ second < 60, "Second is not between 0 and 60.")
        new(hour, minute, second)
    end
end

MyTime

The `struct MyTime` now has four inner constructor methods:
- `MyTime()`
- `MyTime(hour::Int64)`
- `MyTime(hour::Int64, minute::Int64)`
- `MyTime(hour::Int64, minute::Int64, second::Int64)`

An inner constructor method is always defined inside the block of a type declaration, and it has access to a special function called `new` that creates objects of the newly declared type.

In [2]:
t = MyTime(23, 89, 10)

AssertionError: AssertionError: Minute is not between 0 and 60.

In [58]:
t = MyTime(23, 89)

AssertionError: AssertionError: Minute is not between 0 and 60.

In [60]:
t = MyTime(24, 0, 89)

AssertionError: AssertionError: Second is not between 0 and 60.

**Exercise 16-1**  
Write a function called `print_time` that takes a MyTime object and prints it in the form hour:minute:second. The `@printf` macro of the standard library module `Printf` prints an integer with the format sequence "%02d" using at least two digits, including a leading zero if necessary.



In [26]:
using Printf

function print_time(t::MyTime)
    @printf("%02d:%02d:%02d\n", t.hour, t.minute, t.second)
end

print_time (generic function with 1 method)

In [4]:
t = MyTime(10, 11,54) # using default (implicit) constructor

print_time(t)

10:11:54

**Exercise 16-2**  
Write a Boolean function called `is_after` that takes two MyTime objects, t1 and t2, and returns true if t1 follows t2 chronologically and false otherwise. Challenge: don’t use an if statement.

In [5]:
function is_after(t1::MyTime, t2::MyTime)
    return t1.hour < t2.hour || t1.minute < t2.minute || t1.second < t2.second
end

is_after (generic function with 1 method)

In [6]:
(t1, t2) = (MyTime(10, 11, 54), MyTime(11, 11, 54)) 
is_after(t1, t2)


true

In [7]:
(t1, t2) = (MyTime(11, 11, 54), MyTime(11, 12, 54)) 
is_after(t1, t2)

true

In [8]:
(t1, t2) = (MyTime(11, 11, 54), MyTime(11, 11, 55)) 
is_after(t1, t2)

true

In [9]:
(t1, t2) = (MyTime(11, 11, 54), MyTime(11, 11, 54)) 
!is_after(t1, t2) # negation

true

In [10]:
(t1, t2) = (MyTime(12, 11, 54), MyTime(11, 11, 54)) 
!is_after(t1, t2) # negation

true

### Pure Functions

In [13]:
function add_time(t1::MyTime, t2::MyTime)
    """
    Creates a new MyTime object, initializes its fields, and returns a reference to the new object.
    """
    sec  = t1.second + t2.second
    min  = t1.minute + t2.minute
    hour = t1.hour + t2.hour
    
    sec, min  = add_hlpr(sec, min)
    min, hour = add_hlpr(min, hour) 
    
    MyTime(hour, min, sec)
end

function add_hlpr(unit1, unit2, mod=60)
    if unit1 > mod
        unit1 -= 60
        unit2 += 1
    end
    (unit1, unit2)
end

add_hlpr (generic function with 2 methods)

In [14]:
(t1, t2) = (MyTime(12, 11, 54), MyTime(11, 11, 54)) 
t_add = add_time(t1, t2)


MyTime(23, 23, 48)

In [15]:
(t1, t2) = (MyTime(12, 11, 54), MyTime(18, 11, 54)) 
t_add = add_time(t1, t2)


MyTime(30, 23, 48)

### Using Conversions


When we wrote add_time, we were effectively doing addition in base 60, which is why we had to carry from one column to the next.  
This observation suggests another approach to the whole problem — we can convert MyTime objects to integers and take advantage of the fact that the computer knows how to do integer arithmetic.

In [52]:
function time_to_int(t::MyTime)::Int64
  return t.second + t.minute * 60 + t.hour * 3600
end

time_to_int (generic function with 1 method)

In [53]:
function int_to_time(val::Int64)::MyTime
  h = val ÷ 3600
  val %= 3600
  m = val ÷ 60
  s = val % 60 
  return MyTime(h, m, s)
end

int_to_time (generic function with 1 method)

In [54]:
t = MyTime(1, 10, 10)
time_to_int(t)

4210

In [55]:
t = MyTime(2, 31, 10)
print_time(t)

act = time_to_int(t)
exp = 9_070
@assert(act == exp, "3h31m10s is 9070s")

t = MyTime(24, 0, 0)
print_time(t)

act = time_to_int(t)
exp = 86_400
@assert(act == exp, "24h is 86400s")

02:31:10
24:00:00


In [56]:
s = 9075
act = int_to_time(s)
exp = MyTime(2, 31, 15)

@assert(act == exp, """$s should be $(@sprintf("%02d:%02d:%02d", exp.hour, exp.minute, exp.second))""")

In [57]:
s = 86_401
act = int_to_time(s)
exp = MyTime(24, 0, 1)

@assert(act == exp, """$s should be $(@sprintf("%02d:%02d:%02d", exp.hour, exp.minute, exp.second))""")

### Other

In [61]:
function Base.show(io::IO, time::MyTime)
  @printf(io, "%02d:%02d:%02d", time.hour, time.minute, time.second)
end

The prefix `Base` is needed because we want to add a new method to the `Base.show` function.
When you print an object, Julia invokes the `show` function:

In [63]:
time = MyTime(9, 45)

09:45:00

In [64]:
time = MyTime(12, 45, 31)

12:45:31

### Operator overloading

In [66]:
import Base.+

function +(t1::MyTime, t2::MyTime)
  seconds = time_to_int(t1) + time_to_int(t2)
  int_to_time(seconds)
end

+ (generic function with 162 methods)

In [67]:
(t1, t2) = (MyTime(12, 11, 54), MyTime(11, 11, 54)) 
t = t1 + t2 # using Base.show


23:23:48

In [70]:
function +(t1::MyTime, seconds::Int64)
  seconds = time_to_int(t1) + seconds
  int_to_time(seconds)
end

+ (generic function with 163 methods)

In [71]:
(t1, t2) = (MyTime(12, 11, 54), 3600) 
t = t1 + t2 # using Base.show

13:11:54

**Exercise 18-1**  
Write a < method for MyTime objects. You can use tuple comparison, but you also might consider comparing integers.

In [74]:
import Base.isless

function isless(t1::MyTime, t2::MyTime)
    (t1.hour, t1.minute, t1.second) < (t2.hour, t2.minute, t2.second)
end

isless (generic function with 42 methods)

In [75]:
(t1, t2) = (MyTime(11, 11, 54), MyTime(11, 11, 55))

t1 < t2

true

In [76]:
t2 < t1

false

In [77]:
t1 < t1

true

In [79]:
t1 ≤ t2

true

In [80]:
t1 ≤ t1

true